In [1]:
import numpy as np
import random


def transiction(s, a):
    mu = np.sum(np.array(s) + np.array(a))/2 -.5
    return tuple(np.random.normal(mu, 1, 2))
    # return random.gauss(s[0]+a, 1), random.gauss(s[1]+a, 1)

def generate(s, n):    
    for _ in range(n):
        a = random.choice([(1,0), (-1,0), (0,1), (0,-1)])
        # a = (1,0)
        yield s, a
        s = transiction(s, a)


s_0 = (1,1)
O = [(s, a) for (s, a) in generate(s_0, 100)]
O

[((1, 1), (0, 1)),
 ((1.3350983815024324, 2.590775706093865), (1, 0)),
 ((1.672924012658255, 1.7560047470146842), (0, 1)),
 ((3.321432161519451, 0.3430763065502076), (-1, 0)),
 ((-2.3523625241180373, 0.916412411953661), (1, 0)),
 ((-1.729803247088204, 0.4170790600045403), (-1, 0)),
 ((-3.196969117451058, -2.067285037856866), (-1, 0)),
 ((-3.6845141182370296, -2.8331524361203186), (-1, 0)),
 ((-5.2723572313566125, -3.953488220777867), (1, 0)),
 ((-5.7652490263341605, -5.136544954164818), (0, 1)),
 ((-5.818427673948406, -6.879327268568582), (-1, 0)),
 ((-8.18507287586351, -6.527438010950208), (1, 0)),
 ((-6.5492554426695095, -7.822621392451232), (1, 0)),
 ((-8.151178490684359, -6.53716331577074), (-1, 0)),
 ((-7.768247834006073, -7.095339375413927), (-1, 0)),
 ((-6.291987272004995, -10.324685966408273), (0, -1)),
 ((-9.047423533177948, -7.124653528578812), (0, 1)),
 ((-7.9980469245991825, -8.036789609261525), (1, 0)),
 ((-8.286114815653352, -8.494279887672004), (0, 1)),
 ((-7.64091785405

$\cal{L}[\xi] = log \; p_0(s_0) \sum_{t=0}^{\mathrm{T}-1}log \; (\pi_{\theta}(a_t|s_t)p(s_{t+1}|s_t, a_t))$

In [2]:
from scipy.stats import norm

def ll(O, theta):
    p = lambda s_, s, a: np.prod(norm.pdf(s_, np.sum(np.array(s) + np.array(a))/2+ theta, 1))
    return np.sum([np.log(p(s, O[t-1][0], O[t-1][1])) for t,(s,_) in enumerate(O) if 0 < t])

ll(O, -.5)

-298.349590540814

In [3]:
best_theta = 0
best_ll = -np.inf

for i in range(100):
    theta = 0 - i/10
    new_ll = ll(O, theta)
    if new_ll > best_ll:
        best_ll = new_ll
        best_theta = theta

print(best_theta, best_ll)

-0.4 -298.0771323228694


In [5]:
import torch
from torch.autograd import Variable
from torch.distributions import normal

s = Variable(torch.from_numpy(np.array(O)[:-1,0])).type(torch.FloatTensor)
a = Variable(torch.from_numpy(np.array(O)[:-1,1])).type(torch.IntTensor)
s_ = Variable(torch.from_numpy(np.array(O)[1:,0])).type(torch.FloatTensor)


p = Variable(torch.Tensor([-.5]), requires_grad=True)

loss = torch.sum(normal.Normal(torch.sum(s + a, 1)/2 + p, 1).log_prob(s_[:,0]) + normal.Normal(torch.sum(s + a, 1)/2 + p, 1).log_prob(s_[:,1]))
# loss = torch.sum(torch.log(torch.exp(normal.Normal(torch.sum(s + a, 1)/2 + p, 1).log_prob(s_[:,0])) * torch.exp(normal.Normal(torch.sum(s + a, 1)/2 + p, 1).log_prob(s_[:,1]))))

loss.backward()
gradiente = p.grad.clone()
print('like: ', loss)
# gradiente.numpy()

like:  tensor(-298.3496, grad_fn=<SumBackward0>)


In [6]:
import torch
from torch.autograd import Variable
from torch.distributions import normal

s = Variable(torch.from_numpy(np.array(O)[:-1,0])).type(torch.FloatTensor)
a = Variable(torch.from_numpy(np.array(O)[:-1,1])).type(torch.IntTensor)
s_ = Variable(torch.from_numpy(np.array(O)[1:,0])).type(torch.FloatTensor)

p = Variable(torch.randn(1), requires_grad=True)

learning_rate = 0.00002
for t in range(10000):
    NLL = -torch.sum(normal.Normal(torch.sum(s + a, 1)/2 + p, 1).log_prob(s_[:,0]) + normal.Normal(torch.sum(s + a, 1)/2 + p, 1).log_prob(s_[:,1]))
    NLL.backward()
    
    p.data -= learning_rate * p.grad.data
    p.grad.data.zero_()

print("loglike  =", NLL.data.numpy(), "p =", p.data.numpy())

loglike  = 297.9471 p = [-0.43623546]
